In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

kategoriler = []
katalog = []
ürünler = []
urunler_pd = []
soup = BeautifulSoup(requests.get(f'https://www.se.com/tr/tr/product-range-az').content, 'html.parser')

tum_urunler = soup.find('ul', class_='product-finder-result')
olmayan_urunler = tum_urunler.find_all('li' , class_="product-finder-result__item--commercial-status")
olan_urunler = tum_urunler.find_all('li')

olmayan_urun_linkleri = [link['href'] for urun in olmayan_urunler for link in urun.find_all('a')]
olan_urun_linkleri = [link['href'] for urun in olan_urunler for link in urun.find_all('a')]

for link in olan_urun_linkleri:
    if link not in olmayan_urun_linkleri:
        print(link)

    # break
    # if i == j:
    #     print(i)
    # else:
    #     print("bulunamadı")



https://www.se.com/tr/tr/product-range/610-abr-abs/
https://www.se.com/tr/tr/product-range/2571-actassi-opb/
https://www.se.com/tr/tr/product-range/61003-actassi-yap%C4%B1sal-kablolama-%C3%A7%C3%B6z%C3%BCmleri/
https://www.se.com/tr/tr/product-range/61095-acti-9-c60hdc-c60pvdc/
https://www.se.com/tr/tr/product-range/836-acti-9-cds/
https://www.se.com/tr/tr/product-range/61703-acti-9-ic60-lma/
https://www.se.com/tr/tr/product-range/64209-acti-9-ic60-rcbo-2p-3p-4p/
https://www.se.com/tr/tr/product-range/64061-acti-9-ict+/
https://www.se.com/tr/tr/product-range/61083-acti-9-iem2000-serisi/
https://www.se.com/tr/tr/product-range/61273-acti-9-iem3000-serisi/
https://www.se.com/tr/tr/product-range/7565-acti-9-ipb/
https://www.se.com/tr/tr/product-range/61713-acti-9-issw/
https://www.se.com/tr/tr/product-range/61705-acti-9-iswna/
https://www.se.com/tr/tr/product-range/61704-acti-9-ng125-lma/
https://www.se.com/tr/tr/product-range/60180-acti-9-reflex-ic60/
https://www.se.com/tr/tr/product-rang

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json

def extract_json_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    json_element = soup.find('pes-description-and-specifications')
    if json_element:
        json_data = json_element['plain-characteristic-tables']
        return json.loads(json_data)
    return None

def parse_technical_details(ürün_ismi,ürün_linki,stok_kodu, json_data):
    technical_details = []
    for table in json_data:
        table_name = table['tableName']
        for row in table['rows']:
            characteristic_name = row['characteristicName']
            for value in row['characteristicValues']:
                feature = value['labelText']
                technical_details.append({'Ürün İsmi': ürün_ismi, 'Ürün Linki': ürün_linki,'Stok Kodu': stok_kodu, 'Filtre': characteristic_name, 'Özellik': feature})
    return technical_details

def get_details(soup):
    # Ürün ID'sini ve medya bilgilerini al
    product_main = soup.find('pes-product-main')
    product_id = product_main.get('plain-product-id')
    product_media_json = product_main.get('plain-product-media')
    product_cta_area_json = product_main.get('plain-cta-area')

    # JSON verilerini Python sözlüğüne dönüştür
    product_media = json.loads(product_media_json.replace('&quot;', '"'))
    product_cta_area = json.loads(product_cta_area_json.replace('&quot;', '"'))

    # Ürün görsel linklerini ve fiyat bilgisini al
    product_images = product_media.get('zoomPictureDesktop', {}).get('url')
    product_price = product_cta_area.get('pdsPrice')  # Fiyat bilgisini al

    return product_id, product_images, product_price

driver = webdriver.Firefox()
df = pd.read_excel('STANDART TEKLİF FORMATI-V7 FINAL.xlsx')
ürün_kodlari = df['Referans'].to_list()
ürünler = []
teknik_özellikler = []

for ürün_kodu in ürün_kodlari[0:5]:
    ürün_linki = f'https://www.se.com/tr/tr/product/{ürün_kodu}'
    driver.get(ürün_linki)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))


    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    product_id, product_images, product_price = get_details(soup)    

    print("Ürün ID:", product_id)
    print("Ürün Görsel Linkleri:", product_images)
    print("Ürün Fiyatı:", product_price)

    ürün_ismi = driver.title
    ürün_fiyatı = product_price if product_price else ''
    stok_kodu = product_id if product_id else ''

    # 'plain-long-desc-sentences' kısmını çekmek için uygun seçiciyi kullanın
    açıklama_kısmı = soup.find('pes-description-and-specifications')
    if açıklama_kısmı:
        # İlgili kısmı JSON olarak işleyin veya doğrudan metin olarak alın
        ürün_açıklaması = açıklama_kısmı.get('plain-long-desc-sentences')
    else:
        ürün_açıklaması = ''

    json_data = extract_json_data(soup.prettify())
    if json_data:
        teknik_detaylar = parse_technical_details(ürün_ismi, ürün_linki, stok_kodu, json_data)
        teknik_özellikler.extend(teknik_detaylar)

    ürün_fotoları = [product_images]

    ürün = {
        "ÜRÜN İSMİ": ürün_ismi,
        "ÜRÜN FİYATI": ürün_fiyatı,
        "STOK KODU": stok_kodu,
        "URUN ACİKLAMASİ": ürün_açıklaması,
        "FOTOĞRAF LİNKLERİ": ', '.join(ürün_fotoları)
    }
    ürünler.append(ürün)

df_ürünler = pd.DataFrame(ürünler)
df_ürünler.to_excel("Ürünler.xlsx", index=False)

df_teknik = pd.DataFrame(teknik_özellikler)
df_teknik.to_excel("Teknik_Özellikler.xlsx", index=False)

driver.quit()


Ürün ID: 50171
Ürün Görsel Linkleri: https://download.schneider-electric.com/files?p_Doc_Ref=041857&p_File_Type=rendition_1500_jpg
Ürün Fiyatı: None
Ürün ID: 50172
Ürün Görsel Linkleri: https://download.schneider-electric.com/files?p_Doc_Ref=041857&p_File_Type=rendition_1500_jpg
Ürün Fiyatı: None
Ürün ID: 50183
Ürün Görsel Linkleri: https://download.schneider-electric.com/files?p_Doc_Ref=041857&p_File_Type=rendition_1500_jpg
Ürün Fiyatı: None
Ürün ID: 16003
Ürün Görsel Linkleri: https://download.schneider-electric.com/files?p_Doc_Ref=PB101115&p_File_Type=rendition_1500_jpg
Ürün Fiyatı: None
Ürün ID: 16004
Ürün Görsel Linkleri: https://download.schneider-electric.com/files?p_Doc_Ref=PB101115&p_File_Type=rendition_1500_jpg
Ürün Fiyatı: None


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def extract_json_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    json_element = soup.find('pes-description-and-specifications')
    if json_element:
        json_data = json_element['plain-characteristic-tables']
        return json.loads(json_data)
    return None

def parse_technical_details(ürün_ismi, ürün_linki, stok_kodu, json_data):
    technical_details = []
    for table in json_data:
        table_name = table['tableName']
        for row in table['rows']:
            characteristic_name = row['characteristicName']
            for value in row['characteristicValues']:
                feature = value['labelText']
                technical_details.append({'Ürün İsmi': ürün_ismi, 'Ürün Linki': ürün_linki, 'Stok Kodu': stok_kodu, 'Filtre': characteristic_name, 'Özellik': feature})
    return technical_details

def get_details(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Web sayfası yüklenemedi:", url)
        return None, None, None

    soup = BeautifulSoup(response.content, 'html.parser')

    product_main = soup.find('pes-product-main')
    if not product_main:
        return None, None, None

    product_id = product_main.get('plain-product-id')
    product_media_json = product_main.get('plain-product-media')
    product_media = json.loads(product_media_json.replace('&quot;', '"'))
    product_images = product_media.get('zoomPictureDesktop', {}).get('url')

    return soup, product_id, product_images

df = pd.read_excel('STANDART TEKLİF FORMATI-V7 FINAL.xlsx')
ürün_kodlari = df['Referans'].to_list()
ürünler = []
teknik_özellikler = []

for ürün_kodu in ürün_kodlari[0:5]:
    ürün_linki = f'https://www.se.com/tr/tr/product/{ürün_kodu}'
    soup, product_id, product_images = get_details(ürün_linki)

    if not soup:
        continue

    ürün_ismi = soup.title.string if soup.title else ''
    stok_kodu = product_id if product_id else ''
    ürün_fotoları = [product_images] if product_images else []

    json_data = extract_json_data(soup.prettify())
    if json_data:
        teknik_detaylar = parse_technical_details(ürün_ismi, ürün_linki, stok_kodu, json_data)
        teknik_özellikler.extend(teknik_detaylar)

    ürün = {
        "ÜRÜN İSMİ": ürün_ismi,
        "STOK KODU": stok_kodu,
        "FOTOĞRAF LİNKLERİ": ', '.join(ürün_fotoları),
        "ÜRÜN LİNKİ": ürün_linki
    }
    ürünler.append(ürün)

df_ürünler = pd.DataFrame(ürünler)
df_ürünler.to_excel("Ürünler.xlsx", index=False)

df_teknik = pd.DataFrame(teknik_özellikler)
df_teknik.to_excel("Teknik_Özellikler.xlsx", index=False)


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def extract_json_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    json_element = soup.find('pes-description-and-specifications')
    if json_element:
        json_data = json_element['plain-characteristic-tables']
        return json.loads(json_data)
    return None

def parse_technical_details(ürün_ismi, ürün_linki, stok_kodu, json_data):
    technical_details = []
    for table in json_data:
        table_name = table['tableName']
        for row in table['rows']:
            characteristic_name = row['characteristicName']
            for value in row['characteristicValues']:
                feature = value['labelText']
                technical_details.append({'Ürün İsmi': ürün_ismi, 'Ürün Linki': ürün_linki, 'Stok Kodu': stok_kodu, 'Filtre': characteristic_name, 'Özellik': feature})
    return technical_details


def get_details(soup):
    product_main = soup.find('pes-product-main')
    if not product_main:
        print("Ürün ana elementi bulunamadı.")
        return None, None, None

    product_id = product_main.get('plain-product-id')
    product_media_json = product_main.get('plain-product-media')
    product_cta_area_json = product_main.get('plain-cta-area')

    product_media = json.loads(product_media_json.replace('&quot;', '"'))
    product_cta_area = json.loads(product_cta_area_json.replace('&quot;', '"'))

    product_images = product_media.get('zoomPictureDesktop', {}).get('url')
    product_price = product_cta_area.get('pdsPrice')

    return product_id, product_images, product_price

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
df = pd.read_excel('STANDART TEKLİF FORMATI-V7 FINAL.xlsx')
ürün_kodlari = df['Referans'].to_list()
ürünler = []
teknik_özellikler = []

for ürün_kodu in ürün_kodlari:
    print(f"Ürün kodu işleniyor: {ürün_kodu}")
    ürün_linki = f'https://www.se.com/tr/tr/product/{ürün_kodu}'
    print(ürün_linki)
    response = requests.get(ürün_linki, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_id, product_images, product_price = get_details(soup)    

    ürün_ismi = soup.title.string if soup.title else ''
    print(ürün_ismi)
    ürün_fiyatı = product_price if product_price else ''
    stok_kodu = product_id if product_id else ''

    # Ürün açıklamasını ve diğer detayları al
    product_details = soup.find('pes-description-and-specifications')
    ürün_açıklaması = product_details.get('plain-long-desc-sentences', '') if product_details else ''


    json_data = extract_json_data(soup.prettify())
    if json_data:
        teknik_detaylar = parse_technical_details(ürün_kodu, ürün_linki, stok_kodu, json_data)
        teknik_özellikler.extend(teknik_detaylar)

    ürün_fotoları = [product_images] if product_images else []


    ürün = {
        "ÜRÜN İSMİ": ürün_ismi,
        "ÜRÜN FİYATI": ürün_fiyatı,
        "STOK KODU": stok_kodu,
        "URUN ACİKLAMASİ": ürün_açıklaması,
        "FOTOĞRAF LİNKLERİ": ', '.join(ürün_fotoları)
    }

    ürünler.append(ürün)

df_ürünler = pd.DataFrame(ürünler)
df_ürünler.to_excel("SCHNEIDER_Ürünler.xlsx", index=False)

df_teknik = pd.DataFrame(teknik_özellikler)
df_teknik.to_excel("SCHNEIDER_Teknik_Özellikler.xlsx", index=False)


Ürün kodu işleniyor: 50171
https://www.se.com/tr/tr/product/50171
50171 - kartuş Cardew - tip C - 440 V | Schneider Electric TR
Ürün kodu işleniyor: 50172
https://www.se.com/tr/tr/product/50172
50172 - kartuş Cardew - tip C - 660 V | Schneider Electric TR
Ürün kodu işleniyor: 50183
https://www.se.com/tr/tr/product/50183
50183 - kartuş Cardew - tip C - 1000 V | Schneider Electric TR
Ürün kodu işleniyor: 16003
https://www.se.com/tr/tr/product/16003


KeyboardInterrupt: 

# THREADLI

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor

def extract_json_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    json_element = soup.find('pes-description-and-specifications')
    if json_element:
        json_data = json_element['plain-characteristic-tables']
        return json.loads(json_data)
    return None

def parse_technical_details(ürün_ismi, ürün_linki, stok_kodu, json_data):
    technical_details = []
    for table in json_data:
        table_name = table['tableName']
        for row in table['rows']:
            characteristic_name = row['characteristicName']
            for value in row['characteristicValues']:
                feature = value['labelText']
                technical_details.append({'Ürün İsmi': ürün_ismi, 'Ürün Linki': ürün_linki, 'Stok Kodu': stok_kodu, 'Filtre': characteristic_name, 'Özellik': feature})
    return technical_details


def get_details(soup):
    product_main = soup.find('pes-product-main')
    if not product_main:
        print("Ürün ana elementi bulunamadı.")
        return None, None, None

    product_id = product_main.get('plain-product-id')
    product_media_json = product_main.get('plain-product-media')
    product_cta_area_json = product_main.get('plain-cta-area')

    product_media = json.loads(product_media_json.replace('&quot;', '"'))
    product_cta_area = json.loads(product_cta_area_json.replace('&quot;', '"'))

    product_images = product_media.get('zoomPictureDesktop', {}).get('url')
    product_price = product_cta_area.get('pdsPrice')

    return product_id, product_images, product_price

def extract_text(item):
    if item:
        return item.get_text(strip=True)
    return ""


def fetch_breadcrumbs(soup):
    breadcrumbs_element = soup.find('pes-breadcrumbs')
    if breadcrumbs_element:
        breadcrumbs_json = breadcrumbs_element.get('plain-breadcrumbs', '[]')
        breadcrumbs_data = json.loads(breadcrumbs_json.replace('&quot;', '"'))
        return [extract_text(BeautifulSoup(breadcrumb['name'], 'html.parser')) for breadcrumb in breadcrumbs_data]
    return []

df = pd.read_excel('STANDART TEKLİF FORMATI-V7 FINAL.xlsx')
ürün_kodlari = df['Referans'].to_list()
ürünler = []
teknik_özellikler = []

def fetch_product_details(ürün_kodu):
    # print(f"Ürün kodu işleniyor: {ürün_kodu}")
    ürün_linki = f'https://www.se.com/tr/tr/product/{ürün_kodu}'
    # print(ürün_linki)
    response = requests.get(ürün_linki)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_id, product_images, product_price = get_details(soup)    

    ürün_ismi = soup.title.string if soup.title else ''
    ürün_fiyatı = product_price if product_price else ''
    stok_kodu = product_id if product_id else ''

    breadcrumbs = fetch_breadcrumbs(soup)
    # print("Breadcrumb:", " > ".join(breadcrumbs))
    kategoriler = " > ".join(breadcrumbs)

    # Ürün açıklamasını ve diğer detayları al
    product_details = soup.find('pes-description-and-specifications')
    ürün_açıklaması = product_details.get('plain-long-desc-sentences', '') if product_details else ''


    json_data = extract_json_data(soup.prettify())
    if json_data:
        teknik_detaylar = parse_technical_details(ürün_kodu, ürün_linki, stok_kodu, json_data)
        # teknik_özellikler.extend(teknik_detaylar)

    ürün_fotoları = [product_images] if product_images else []


    ürün = {
        "ÜRÜN İSMİ": ürün_ismi,
        "ÜRÜN FİYATI": ürün_fiyatı,
        "STOK KODU": stok_kodu,
        "Kategoriler": kategoriler,
        "URUN ACİKLAMASİ": ürün_açıklaması,
        "FOTOĞRAF LİNKLERİ": ', '.join(ürün_fotoları)
    }

    # print('ÜRÜN: ', ürün)

    return ürün, teknik_detaylar

işlenen_ürün_sayısı = 0

with ThreadPoolExecutor(max_workers=20) as executor:
    future_to_ürün_kodu = {executor.submit(fetch_product_details, ürün_kodu): ürün_kodu for ürün_kodu in ürün_kodlari}
    for future in future_to_ürün_kodu:
        ürün, teknik_detaylar = future.result()
        ürünler.append(ürün)
        teknik_özellikler.extend(teknik_detaylar)

        işlenen_ürün_sayısı += 1  # Sayacı artır
        print(f"İşlenen ürün: {işlenen_ürün_sayısı} / Toplam ürün: {len(ürün_kodlari)}")

df_ürünler = pd.DataFrame(ürünler)
df_ürünler.to_excel("SCHNEIDER_Ürünler.xlsx", index=False)

df_teknik = pd.DataFrame(teknik_özellikler)
df_teknik.to_excel("SCHNEIDER_Teknik_Özellikler.xlsx", index=False)


C:\Users\furka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


İşlenen ürün: 1 / Toplam ürün: 30811
İşlenen ürün: 2 / Toplam ürün: 30811
İşlenen ürün: 3 / Toplam ürün: 30811
İşlenen ürün: 4 / Toplam ürün: 30811
İşlenen ürün: 5 / Toplam ürün: 30811
İşlenen ürün: 6 / Toplam ürün: 30811
Ürün ana elementi bulunamadı.
İşlenen ürün: 7 / Toplam ürün: 30811
İşlenen ürün: 8 / Toplam ürün: 30811
İşlenen ürün: 9 / Toplam ürün: 30811
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
İşlenen ürün: 10 / Toplam ürün: 30811
İşlenen ürün: 11 / Toplam ürün: 30811
İşlenen ürün: 12 / Toplam ürün: 30811
İşlenen ürün: 13 / Toplam ürün: 30811
İşlenen ürün: 14 / Toplam ürün: 30811
İşlenen ürün: 15 / Toplam ürün: 30811
İşlenen ürün: 16 / Toplam ürün: 30811
İşlenen ürün: 17 / Toplam ürün: 30811
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürün ana elementi bulunamadı.
Ürü

In [1]:
from rich.progress import Progress
import time

def download_simulation():
    # 100 adımlık bir görevi simüle edelim
    with Progress() as progress:
        task = progress.add_task("[green]İndiriliyor...", total=100)
        while not progress.finished:
            # Görevi belirli bir süre boyunca simüle edelim
            progress.update(task, advance=1)
            time.sleep(0.05)

# Simülasyonu çalıştır
download_simulation()


C:\Users\furka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\P
ython39\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')